# Wav to data convert

In [3]:
import librosa
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Audio
from librosa import display
import numpy as np
import os
import seaborn as sns
from pathlib import Path

from pydub import AudioSegment

import warnings
warnings.filterwarnings('ignore')#safety is no one priority

In [4]:
data_folder = Path("E:/DS/Projects/tunes_repo/MIRaGE/Song_repo/")

music_list = os.listdir(data_folder/'Mirage_wav')
music_wav = "E:/DS/Projects/tunes_repo/MIRaGE/Song_repo/Mirage_wav"

In [5]:
music_list

['AC DC_back in Black_classic_rock.classic_metal.pop_rock.wav',
 'AC DC_Hells bells_classic_rock.classic_metal.pop_rock.wav',
 'AC DC_Highway to hell_classic_rock.classic_metal.pop_rock.wav',
 'AC DC_shook me all night long_classic_rock.classic_metal.pop_rock.wav',
 'AC DC_Thunderstuck_classic_rock.classic_metal.pop_rock.wav',
 'Black sabbath_children of the grave_heavy_metal.black_metal.classic_metal.wav',
 'Black sabbath_god is dead_heavy_metal.black_metal.classic_metal.wav',
 'Black sabbath_Iron man_heavy_metal.black_metal.classic_metal.wav',
 'Black sabbath_paranoid_heavy_metal.black_metal.classic_metal.wav',
 'Black sabbath_war pigs_heavy_metal.black_metal.classic_metal.wav',
 'Children of Bodom_Angels donít kill_heavy_metal.black_metal.wav',
 'Children of Bodom_Are you dead yet_heavy_metal.black_metal.wav',
 'Children of Bodom_Everytime I die_heavy_metal.black_metal.wav',
 'Children of Bodom_Hate crew deathroll_heavy_metal.black_metal.wav',
 'Children of Bodom_Silent night_heavy_

In [6]:
features = []
for track_num in range(0, len(music_list)):    
    band = music_list[track_num].split('_')[0]
    track = music_list[track_num].split('_')[1]
#     genres = ['black_metal','heavy_metal','classic_metal','progressive_metal','progressive_rock',
#               'psychadelic_rock','classic_rock','hard_rock','pop_rock','punk_rock']

    genres = music_list[track_num].split('.')
    genr = 0
    genres_ = []
    for genr in range(0, len(genres)-1):
        if genr == 0:
            genre = [music_list[track_num].split('.')[genr].split('_')[-2] +'_'+ music_list[track_num].split('.')[genr].split('_')[-1]]
        else:
            genre = [music_list[track_num].split('.')[genr]]
        genres_.extend(genre)
        
    features_0 = [band, track, genres_]
    features.append(features_0)

In [7]:
data = pd.DataFrame()

for song in music_list:
    print('starting song ' + song)
    song_ctr = 0       

    curr_track = music_wav + "/" + str(song)
    y, sr = librosa.load(curr_track)
    
    oa_ctr = 0
    
    #song features
    
    Genre = features[song_ctr][2]
    Band = features[song_ctr][0]
    
    song_ctr= song_ctr+1
    
    song_tempo, song_beats = librosa.beat.beat_track(y=y, sr=sr)
    song_zcr = np.median(librosa.feature.zero_crossing_rate(y))
    song_spec_cent = np.median(librosa.feature.spectral_centroid(y=y, sr=sr))
    song_spec_bw = np.median(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    song_rolloff = np.median(librosa.feature.spectral_rolloff(y=y, sr=sr))
    
    try:
        song_tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
        song_chroma = librosa.feature.chroma_cqt(y,sr)
    except:
        song_tonnetz = np.zeros(len(song)).reshape((6,int(len(song)/6)))        
        song_chroma = np.zeros(len(song)).reshape((12,int(len(song)/12))) 

    song_tonnetz_median = []
    for c in song_tonnetz:
        song_tonnetz_median.append(np.median(c))

    song_tonal_features = np.array(song_tonnetz_median)

    song_chroma_median = []
    for c in song_chroma:
        song_chroma_median.append(np.median(c))

    song_beat_features = np.array(song_chroma_median)
    
    
    
     #Cut snippet    
        
    top_db = np.percentile(librosa.core.amplitude_to_db(y, ref=0),5)
    snippets = librosa.effects.split(y, top_db=top_db, frame_length=2048, hop_length=512)
    
    for snippet_count in range(0,len(snippets)-1):
        
        print('Starting song Snippet ' + str(snippet_count), end = '\r')
        
        start = snippets[snippet_count][0]
        end = snippets[snippet_count][1]
        snippet = y[start:end]
        
        #Overall snippet features
        
        snippet_tempo, snippet_beats = librosa.beat.beat_track(y=snippet, sr=sr)
        snippet_count = snippet_count
        snippet_zcr = np.median(librosa.feature.zero_crossing_rate(snippet))
        snippet_rolloff = np.median(librosa.feature.spectral_rolloff(y=snippet, sr=sr))
        snippet_spec_cent = np.median(librosa.feature.spectral_centroid(y=snippet, sr=sr))
        snippet_spec_bw = np.median(librosa.feature.spectral_bandwidth(y=snippet, sr=sr))

        #Overall snippet features- tonnetz, chroma
        
        try:
            snip_tonnetz = librosa.feature.tonnetz(y=snippet, sr=sr)
            snip_chroma = librosa.feature.chroma_cqt(snippet,sr)
        except:
            snip_tonnetz = np.zeros(len(snippet)).reshape((6,int(len(snippet)/6)))        
            snip_chroma = np.zeros(len(snippet)).reshape((12,int(len(snippet)/12))) 
        
        snippet_tonnetz_median = []
        for c in snip_tonnetz:
            snippet_tonnetz_median.append(np.median(c))

        snippet_tonal_features = np.array(snippet_tonnetz_median)

        snippet_chroma_median = []
        for c in snip_chroma:
            snippet_chroma_median.append(np.median(c))
            
        snippet_beat_features = np.array(snippet_chroma_median)


        i = 0
        ctr = 0
        while i < (len(y)-5521):
            start = i
            ctr = ctr+1
            oa_ctr = oa_ctr+1
            end = i+5520

            segment = y[start:end]


            tempo, beats = librosa.beat.beat_track(y=segment, sr=sr)
            onset_strength = np.median(librosa.onset.onset_strength(y=y, sr=sr))

            spec_cent = np.median(librosa.feature.spectral_centroid(y=segment, sr=sr))
            spec_bw = np.median(librosa.feature.spectral_bandwidth(y=segment, sr=sr))
            rolloff = np.median(librosa.feature.spectral_rolloff(y=segment, sr=sr))
            zcr = np.median(librosa.feature.zero_crossing_rate(segment))
            
            
            try:
                tonnetz = librosa.feature.tonnetz(y=segment, sr=sr)
                chroma = librosa.feature.chroma_cqt(segment,sr)
            except:
                tonnetz = np.zeros(len(len)).reshape((6,int(len(segment)/6)))        
                chroma = np.zeros(len(segment)).reshape((12,int(len(segment)/12))) 

            tonnetz_median = []
            for c in tonnetz:
                tonnetz_median.append(np.median(c))

            tonal_features = np.array(tonnetz_median)

            chroma_median = []
            for c in chroma:
                chroma_median.append(np.median(c))

            beat_features = np.array(snippet_chroma_median)
            beat_count = ctr
            oa_beat_count = oa_ctr
            seg_mean = np.absolute(segment).mean()


            df_list = [Band, Genre, tempo, spec_cent, spec_bw, rolloff, zcr, onset_strength, beat_features, tonal_features,
                       snippet_tempo, snippet_spec_cent, snippet_spec_bw, snippet_rolloff, zcr, snippet_beat_features, snippet_tonal_features,
                       song_tempo, song_spec_cent, song_spec_bw, song_rolloff, song_zcr,song_beat_features, song_tonal_features,
                       oa_ctr, snippet_count, beat_count,seg_mean]
                       
            data = data.append(pd.Series(df_list),ignore_index=True)

            i = end
            
        print("completed song Snippet " + str(snippet_count), end = "\r")
        print("data_points " + str(oa_ctr), end = "\r")
    print("completed song" + song)

print("conversion Successful")

starting song AC DC_back in Black_classic_rock.classic_metal.pop_rock.wav
completed songAC DC_back in Black_classic_rock.classic_metal.pop_rock.wav
starting song AC DC_Hells bells_classic_rock.classic_metal.pop_rock.wav
completed songAC DC_Hells bells_classic_rock.classic_metal.pop_rock.wav
starting song AC DC_Highway to hell_classic_rock.classic_metal.pop_rock.wav
completed songAC DC_Highway to hell_classic_rock.classic_metal.pop_rock.wav
starting song AC DC_shook me all night long_classic_rock.classic_metal.pop_rock.wav
completed songAC DC_shook me all night long_classic_rock.classic_metal.pop_rock.wav
starting song AC DC_Thunderstuck_classic_rock.classic_metal.pop_rock.wav
completed songAC DC_Thunderstuck_classic_rock.classic_metal.pop_rock.wav
starting song Black sabbath_children of the grave_heavy_metal.black_metal.classic_metal.wav
completed songBlack sabbath_children of the grave_heavy_metal.black_metal.classic_metal.wav
starting song Black sabbath_god is dead_heavy_metal.black_

KeyboardInterrupt: 

In [19]:
data

(27365, 26)

In [8]:
data.columns = ['Band', 'Genre', 'tempo', 'spec_cent', 'spec_bw', 'rolloff', 'zcr', 'onset_strength', 'beat_features', 
                'tonal_features','snippet_tempo', 'snippet_spec_cent', 'snippet_spec_bw', 'snippet_rolloff', 'snippet_zcr',
                'snippet_beat_features', 'snippet_tonal_features','song_tempo', 'song_spec_cent', 'song_spec_bw', 'song_neat_features','song_tonal_features',
                'song_rolloff', 'song_zcr', 'oa_ctr', 'snippet_count', 'beat_count','seg_mean']

In [10]:
data.tail()

,Band,Genre,tempo,spec_cent,spec_bw,rolloff,zcr,onset_strength,beat_features,tonal_features,...,song_spec_cent,song_spec_bw,song_neat_features,song_tonal_features,song_rolloff,song_zcr,oa_ctr,snippet_count,beat_count,seg_mean
29758,AC DC,"[classic_rock, classic_metal, pop_rock]",198.768029,2450.415056,2536.666860,5221.801758,0.096680,0.851725,"[0.6701960953451666, 0.43486461035179924, 0.47...","[0.03249354414874142, -0.005748096158925264, 0...",...,2258.397861,2351.810215,4737.304688,0.093262,"[0.422802180831214, 0.442133392730812, 0.50878...","[0.017189908763257725, -0.006011476650654961, ...",8330.0,4.0,746.0,0.131904
29759,AC DC,"[classic_rock, classic_metal, pop_rock]",135.999178,2937.392273,2683.005533,6158.496094,0.150391,0.851725,"[0.6701960953451666, 0.43486461035179924, 0.47...","[0.014835293950495477, 0.01688009662323918, -0...",...,2258.397861,2351.810215,4737.304688,0.093262,"[0.422802180831214, 0.442133392730812, 0.50878...","[0.017189908763257725, -0.006011476650654961, ...",8331.0,4.0,747.0,0.089903
29760,AC DC,"[classic_rock, classic_metal, pop_rock]",198.768029,2815.544641,2866.608459,6642.993164,0.099609,0.851725,"[0.6701960953451666, 0.43486461035179924, 0.47...","[0.009607800573806028, -0.008362329391315685, ...",...,2258.397861,2351.810215,4737.304688,0.093262,"[0.422802180831214, 0.442133392730812, 0.50878...","[0.017189908763257725, -0.006011476650654961, ...",8332.0,4.0,748.0,0.093520
29761,AC DC,"[classic_rock, classic_metal, pop_rock]",198.768029,2582.448422,2636.335766,5609.399414,0.097168,0.851725,"[0.6701960953451666, 0.43486461035179924, 0.47...","[0.0047137344777076215, -0.02579290490177843, ...",...,2258.397861,2351.810215,4737.304688,0.093262,"[0.422802180831214, 0.442133392730812, 0.50878...","[0.017189908763257725, -0.006011476650654961, ...",8333.0,4.0,749.0,0.122746
29762,AC DC,"[classic_rock, classic_metal, pop_rock]",0.000000,2362.002863,2765.310010,5749.365234,0.105469,0.851725,"[0.6701960953451666, 0.43486461035179924, 0.47...","[0.01112363425932702, 0.005774994460304597, 0....",...,2258.397861,2351.810215,4737.304688,0.093262,"[0.422802180831214, 0.442133392730812, 0.50878...","[0.017189908763257725, -0.006011476650654961, ...",8334.0,4.0,750.0,0.046166


In [1]:
#data.to_csv('Song_master.csv')
data.to_pickle("./dummy.pkl")

NameError: name 'data' is not defined